## Scraping Plant's Scientific Name and Link to their details

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
url = "https://neist.res.in/osadhi/chemo.php"
# get headers from "https://www.whatismybrowser.com/"
HEADERS = ({'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

In [ ]:
webpage = requests.get(url, headers=HEADERS)
soup = BeautifulSoup(webpage.content, "html.parser")

In [ ]:
chemo_name_lst = soup.find_all("a", href=True)
chemo_name_lst = chemo_name_lst[12:-9] # slice out unwanted items from the list
chemo_name_lst

In [ ]:
len(chemo_name_lst)

In [ ]:
chem_names_chemo_dict = {'Phytochemical': [], 'Link': []}

In [ ]:
for chemo_name in chemo_name_lst:
    try:
        chem_names_chemo_dict['Phytochemical'].append(chemo_name.text.strip()) # add scientific name of the plant to dict 
        element = chemo_name.find("a", href=True)
        final_link = str("https://neist.res.in/osadhi/" + chemo_name['href'])
        chem_names_chemo_dict['Link'].append(final_link) # add link of the corresponding plant to dict 
    except TypeError:
        pass

In [ ]:
chem_names_chemo_df = pd.DataFrame(chem_names_chemo_dict)
chem_names_chemo_df.head()

In [ ]:
chem_names_chemo_df.to_csv("OSADHI_Chemo_ChemName_Links.csv", index=False)

# Scraping each plant's specific details

In [58]:
soup_man = """
<!DOCTYPE html>
"""

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from tqdm import tqdm
import json

In [64]:
HEADERS = ({'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

In [ ]:
url2 = "https://neist.res.in/osadhi/chemodetail.php?phyto=%0Adicerandrol+C"
webpage2 = requests.get(url2, headers=HEADERS)
soup2 = BeautifulSoup(webpage2.content, "html.parser")

In [ ]:
main_dropdowns = soup2.find_all("div", class_='card-header')

for i in main_dropdowns:
    print(i.text.strip())

In [59]:
chemo_items_dict = {
'Physiochemical Properties' : {'Molecular Weight':'', 'nRot':'', 'Heavy Atom Molecular Weight':'', 'nRig':'', 'Exact Molecular Weight':'', 'nRing':'', 'Solubility: LogS':'', 'nHRing':'', 'Solubility: LogP':'', 'No. of Aliphatic Rings':'', 'Acid Count':'', 'No. of Aromatic Rings':'', 'Base Count':'', 'No. of Aliphatic Carbocycles Rings':'', 'Atoms Count':'', 'No. of Aliphatic Hetero Cycles':'', 'No. of Heavy Atom':'', 'No. of Aromatic Carbocycles':'', 'nHetero':'', 'No. of Aromatic Hetero Cycles':'', 'nBridge Head':'', 'No. Saturated Carbocycles':'', 'No. of Hydrogen atom':'', 'No. of Saturated Hetero Cycles':'', 'No. of Carbon atom':'', 'No. of Saturated Rings':'', 'No. of Nitrogen atom':'', 'No. of Arom Atom':'', 'No. of Oxygen atom':'', 'No. of Arom Bond':'', 'nHA':'', 'APOL':'', 'nHD':'', 'BPOL':''},
'Medicinal Chemistry Properties' : {'QED':'', 'Synth':'', 'Natural Product Likeliness':'', 'NR-PPAR-gamma':''},
'Drug Likeliness' : {'Lipinski':'', 'Pfizer':'', 'GSK':'', 'Golden Triangle':''},
'Absorption' : {'Pgp-inh':'', 'Pgp-sub':'', 'HIA':'', 'CACO-2':''},
'Distribution' : {'MDCK':'', 'BBB':'', 'PPB':'', 'VDSS':''},
'Metabolism' : {'FU':'', 'CYP1A2-inh':'', 'CYP1A2-sub':'', 'CYP2c19-inh':'', 'CYP2c19-sub':'', 'CYP2c9-inh':'', 'CYP2c9-sub':'', 'CYP2d6-inh':'', 'CYP2d6-sub':'', 'CYP3a4-inh':'', 'CYP3a4-sub':''},
'Excretion' : {'CL':'', 'T12':''},
'Toxicity' : {'hERG':'', 'Ames':'', 'ROA':'', 'SkinSen':'', 'Carcinogencity':'', 'EI':'', 'Respiratory':'', 'NR-Aromatase':''},
'Antiviral Prediction' : {'Antiviral':'', 'Prediction':''}
}

In [60]:
physicochemical_properties = ['Molecular Weight', 'nRot', 'Heavy Atom Molecular Weight', 'nRig', 'Exact Molecular Weight', 'nRing', 'Solubility: LogS', 'nHRing', 'Solubility: LogP', 'No. of Aliphatic Rings', 'Acid Count', 'No. of Aromatic Rings', 'Base Count', 'No. of Aliphatic Carbocycles Rings', 'Atoms Count', 'No. of Aliphatic Hetero Cycles', 'No. of Heavy Atom', 'No. of Aromatic Carbocycles', 'nHetero', 'No. of Aromatic Hetero Cycles', 'nBridge Head', 'No. Saturated Carbocycles', 'No. of Hydrogen atom', 'No. of Saturated Hetero Cycles', 'No. of Carbon atom', 'No. of Saturated Rings', 'No. of Nitrogen atom', 'No. of Arom Atom', 'No. of Oxygen atom', 'No. of Arom Bond', 'nHA', 'APOL', 'nHD', 'BPOL']
medicinal_chemistry_properties = ['QED', 'Synth', 'Natural Product Likeliness', 'NR-PPAR-gamma']
drug_likeliness = ['Lipinski', 'Pfizer', 'GSK', 'Golden Triangle']
absorption = ['Pgp-inh', 'Pgp-sub', 'HIA', 'CACO-2']
distribution = ['MDCK', 'BBB', 'PPB', 'VDSS']
metabolism = ['FU', 'CYP1A2-inh', 'CYP1A2-sub', 'CYP2c19-inh', 'CYP2c19-sub', 'CYP2c9-inh', 'CYP2c9-sub', 'CYP2d6-inh', 'CYP2d6-sub', 'CYP3a4-inh', 'CYP3a4-sub']
excretion = ['CL', 'T12']
toxicity = ['hERG', 'Ames', 'ROA', 'SkinSen', 'Carcinogencity', 'EI', 'Respiratory', 'NR-Aromatase']
antiviral_prediction = ['Antiviral', 'Prediction']

In [61]:
def add_details_to_dict(sub_header_lst):
    k=0
    for i in sub_header_lst:
        if k % 2 == 0: # heading
            heading = i.text.strip()
        else:          # values
            if heading in physicochemical_properties:
                chemo_items_dict['Physiochemical Properties'][heading] = str(i.text.strip())
            
            elif heading in medicinal_chemistry_properties:
                chemo_items_dict['Medicinal Chemistry Properties'][heading] = str(i.text.strip())
            
            elif heading in drug_likeliness:
                chemo_items_dict['Drug Likeliness'][heading] = str(i.text.strip())
            
            elif heading in absorption:
                chemo_items_dict['Absorption'][heading] = str(i.text.strip())
            
            elif heading in distribution:
                chemo_items_dict['Distribution'][heading] = str(i.text.strip())
            
            elif heading in metabolism:
                chemo_items_dict['Metabolism'][heading] = str(i.text.strip())
            
            elif heading in excretion:
                chemo_items_dict['Excretion'][heading] = str(i.text.strip())
            
            elif heading in toxicity:
                chemo_items_dict['Toxicity'][heading] = str(i.text.strip())
            
            elif heading in antiviral_prediction:
                chemo_items_dict['Antiviral Prediction'][heading] = str(i.text.strip())
        k+=1
    return chemo_items_dict


def check_sub_headers(sub_header_html):
    # sub_header_values_lst = []
    
    # j=0
    # for i in sub_header_html:
    #     if j%2 != 0:
    #         sub_header_values_lst.append(i.text.strip())
    #     j+=1
    # print(len(sub_header_values_lst)) 
    # print(sub_header_values_lst) 
    if len(sub_header_html) == 0:
        return "No" # no sub headers
    else:
        val_dict = add_details_to_dict(sub_header_html)
        return ("Yes", val_dict) # sub_header_values_lst # pass to subsequent function

# def get_website_html(plant_link):
#     webpage = requests.get(plant_link, headers=HEADERS)
#     soup = BeautifulSoup(webpage.content, "html.parser")
#     return soup
    

In [65]:
url2 = "https://neist.res.in/osadhi/chemodetail.php?phyto=%0Adicerandrol+C"
url3 = "https://neist.res.in/osadhi/chemodetail.php?phyto=+%281S%2C8%27R%29-6-methoxy-2-methylspiro%5B3%2C4-dihydroisoquinoline-1%2C7%27-6%2C8-dihydrocyclopenta%5Bg%5D%5B1%2C3%5Dbenzodioxole%5D-7%2C8%27-diol"
url4 = "https://neist.res.in/osadhi/chemodetail.php?phyto=%0AVitexin+2%27%27-O-beta-L-rhamnoside"
webpage3 = requests.get(url2, headers=HEADERS)
soup3 = BeautifulSoup(webpage3.content, "html.parser")

# soup_man2 = BeautifulSoup(soup_man, "html.parser")
# main_headers = soup_man2.find_all("div", class_='card-header')

main_headers = soup3.find_all("div", class_='card-header')

if not main_headers: # No main headers like  Physiochemical Properties, Metabolism etc
    print("main header - no")
elif main_headers: # Main headers present but no content in sub-headers
    print("main header - yes")
    sub_headers = soup_man2.find_all("td")
    reply, sub_headers_with_values = check_sub_headers(sub_headers)
    print(reply,"\n", sub_headers_with_values)

ConnectionError: HTTPSConnectionPool(host='neist.res.in', port=443): Max retries exceeded with url: /osadhi/chemodetail.php?phyto=%0Adicerandrol+C (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7effb9960f40>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))

In [44]:
len(soup_man2.find_all("td"))

146

In [ ]:
soup2.find("div", class_='card-body')

In [ ]:
soup2.find_all('td')

In [ ]:
chemo_values = []

In [ ]:
inside_info = soup2.find_all('td')

j=0
for i in inside_info:
    if j%2 != 0:
        chemo_values.append(i.text.strip())
    j+=1

In [ ]:
for i in lst:
    if i % 2 == 0: # heading
        heading = i.text.strip()
    else:          # values
        if heading in physicochemical_properties:
            chemo_items_dict['Physiochemical Properties'][heading] = str(i.text.strip())
        
        elif heading in medicinal_chemistry_properties:
            chemo_items_dict['Medicinal Chemistry Properties'][heading] = str(i.text.strip())
        
        elif heading in drug_likeliness:
            chemo_items_dict['Drug Likeliness'][heading] = str(i.text.strip())
        
        elif heading in absorption:
            chemo_items_dict['Absorption'][heading] = str(i.text.strip())
        
        elif heading in distribution:
            chemo_items_dict['Distribution'][heading] = str(i.text.strip())
        
        elif heading in metabolism:
            chemo_items_dict['Metabolism'][heading] = str(i.text.strip())
        
        elif heading in excretion:
            chemo_items_dict['Excretion'][heading] = str(i.text.strip())
        
        elif heading in toxicity:
            chemo_items_dict['Toxicity'][heading] = str(i.text.strip())
        
        elif heading in antiviral_prediction:
            chemo_items_dict['Antiviral Prediction'][heading] = str(i.text.strip())

In [ ]:
all_plant_info_dict = {}

def get_plant_website_html(plant_link):
    plant_webpage = requests.get(plant_link, headers=HEADERS)
    plant_soup = BeautifulSoup(plant_webpage.content, "html.parser")
    return plant_soup

def add_per_plant_details(soup, per_plant_details_dict):
    per_plant_details_dict = {
        'Summary': [],
        'Statewise availability': [],
        'Phytochemicals': [],
        'Ailments cured': [],
        'Plant parts and method of its use': [],
        'Vernacular name': []
        }
    
    for i in range(0, 6):
        if i == 0:
            for j in soup.table.find_all('ul')[i]:
                per_plant_details_dict['Summary'].append(j.text)
        if i == 1:
            for j in soup.table.find_all('ul')[i]:
                per_plant_details_dict['Statewise availability'].append(j.text)
        if i == 2:
            for j in soup.table.find_all('ul')[i]:
                per_plant_details_dict['Phytochemicals'].append(j.text)
        if i == 3:
            for j in soup.table.find_all('ul')[i]:
                per_plant_details_dict['Ailments cured'].append(j.text)
        if i == 4:
            for j in soup.table.find_all('ul')[i]:
                per_plant_details_dict['Plant parts and method of its use'].append(j.text)
        if i == 5:
            for j in soup.table.find_all('ul')[i]:
                per_plant_details_dict['Vernacular name'].append(j.text)

    return per_plant_details_dict

In [ ]:
df = pd.read_csv("OSADHI_Ethno_Scientific_Name_Links.csv")
df.head(3)

In [ ]:
loop_range = range(1494, len(df['Scientific Name']))
big_pause = 0

for i in tqdm((loop_range), desc="Processing"):
    plant_name = df['Scientific Name'][i]
    plant_link = df['Link'][i]

    plant_website_html = get_plant_website_html(plant_link) # getting HTML of plant details website
    per_plant_details = add_per_plant_details(plant_website_html, all_plant_info_dict) # collecting plant info to a dict
    all_plant_info_dict[plant_name] = per_plant_details # adding collected plant info to a main dict with plant sci name as key
    
    big_pause += 1
    if big_pause % 50 == 0:
        time.sleep(2) # after continuous 50 runs, long break for server

In [ ]:
with open("OSADHI_all_plant_details_json_file_NEW.json", "w") as FinalOut: 
    json.dump(all_plant_info_dict, FinalOut)

In [69]:
abc = None

if abc:
    print('yes')
else:
    print('no')

no


# Merging JSONs

In [ ]:
with open('OSADHI_all_plant_details_json_file_1.json') as f:
    data1 = json.load(f)
    
with open('OSADHI_all_plant_details_json_file_NEW.json') as f: 
    data2 = json.load(f)

data1.update(data2)

with open('OSADHI_all_plant_details_json_file.json', 'w') as f:
   json.dump(data1, f)

In [ ]:
with open('OSADHI_all_plant_details_json_file.json') as f:
    item_dict = json.load(f)

len(item_dict)

In [ ]:
sci_name_lst = df['Scientific Name'].tolist()

In [ ]:
for key, _ in item_dict.items():
    try:
        sci_name_lst.remove(key)
    except ValueError:
        pass

In [ ]:
sci_name_lst

In [ ]:
item_dict

# Converting JSON to CSV

In [ ]:
import json
import pandas as pd

In [ ]:
with open('OSADHI_all_plants.json') as f:
    item_dict = json.load(f)

In [ ]:
# item_dict's SUMMARY column DataFrame (df) 

first_entry = list(item_dict.values())[0]
headers = [item.split(':')[0] for item in first_entry['Summary']]  # use first entry to get headers
# headers expected output ['Scientific Name', 'Genus', 'Species', 'Family', 'Synonym']

rows = []

for plant_name, plant_data in item_dict.items():
    summary = plant_data.get('Summary', []) # extracting Summary values
    row = {}
    for idx, header in enumerate(headers):
        # Match summary items with corresponding headers
        row[header] = summary[idx].split(':')[1] if idx < len(summary) else ''

    rows.append(row)

df = pd.DataFrame(rows, columns=headers)
df.head()

In [ ]:
# item_dict's remaining column DataFrame (df1)

df1 = pd.DataFrame.from_dict(item_dict, orient='index')
df1 = df1.map(lambda x: ', '.join(x) if isinstance(x, list) else x)
df1.drop('Summary', axis=1, inplace=True)
df1.head()

In [ ]:
# merging both dfs, df and df1

merged_df = pd.concat([df.reset_index(drop=True), df1.reset_index(drop=True)], axis=1)
merged_df.head()

In [ ]:
merged_df.to_csv("OSADHI_ethno_all_plant_details.csv", index=False)

# Converting JSON to CSV (for nested dictionary)

In [137]:
with open('chemo.json') as f: 
    data = json.load(f)

In [147]:
# new_dict = {}
all_dict = {}

for k, v in data.items():
    # print(k, v)
    new_dict = {}
    for ke, va in v.items():
        # print(ke, va)
        for key, val in va.items():
            # print(f"{ke}_{key} : {val}")
            new_dict[f"{ke} {key}"] = val
    all_dict[k] = new_dict

df = pd.DataFrame(all_dict).T
df.head()

,Physiochemical Properties Molecular Weight,Physiochemical Properties nRot,Physiochemical Properties Heavy Atom Molecular Weight,Physiochemical Properties nRig,Physiochemical Properties Exact Molecular Weight,Physiochemical Properties nRing,Physiochemical Properties Solubility: LogS,Physiochemical Properties nHRing,Physiochemical Properties Solubility: LogP,Physiochemical Properties No. of Aliphatic Rings,...,Toxicity hERG,Toxicity Ames,Toxicity ROA,Toxicity SkinSen,Toxicity Carcinogencity,Toxicity EI,Toxicity Respiratory,Toxicity NR-Aromatase,Antiviral Prediction Antiviral,Antiviral Prediction Prediction
auricularic acid,302.458,2,272.218,19,302.225,3,-4.698,0,4.505,3,...,0.007,0.015,0.3,0.065,0.107,0.026,0.904,0.012,No,0.54248
d-glucose,180.156,1,168.06,6,180.063,1,-0.017,1,-2.499,1,...,0.039,0.203,0.139,0.04,0.013,0.025,0.03,0.004,No,0.893736
EUDESMANE,208.389,1,180.165,11,208.219,2,-6.093,0,5.564,2,...,0.033,0.019,0.029,0.483,0.039,0.948,0.293,0.006,Yes,0.834177
orientin,448.38,3,428.22,24,448.101,4,-3.741,2,-0.067,1,...,0.024,0.822,0.099,0.342,0.035,0.061,0.036,0.943,Yes,0.879454
oxalic acid,90.034,0,88.018,2,89.9953,0,0.24,0,-0.735,0,...,0.024,0.026,0.038,0.504,0.014,0.994,0.068,0.005,No,0.933


In [148]:
df.shape

(20776, 73)

In [149]:
df = df.rename_axis('Phytochemical').reset_index()
df.head()

,Phytochemical,Physiochemical Properties Molecular Weight,Physiochemical Properties nRot,Physiochemical Properties Heavy Atom Molecular Weight,Physiochemical Properties nRig,Physiochemical Properties Exact Molecular Weight,Physiochemical Properties nRing,Physiochemical Properties Solubility: LogS,Physiochemical Properties nHRing,Physiochemical Properties Solubility: LogP,...,Toxicity hERG,Toxicity Ames,Toxicity ROA,Toxicity SkinSen,Toxicity Carcinogencity,Toxicity EI,Toxicity Respiratory,Toxicity NR-Aromatase,Antiviral Prediction Antiviral,Antiviral Prediction Prediction
0,auricularic acid,302.458,2,272.218,19,302.225,3,-4.698,0,4.505,...,0.007,0.015,0.3,0.065,0.107,0.026,0.904,0.012,No,0.54248
1,d-glucose,180.156,1,168.06,6,180.063,1,-0.017,1,-2.499,...,0.039,0.203,0.139,0.04,0.013,0.025,0.03,0.004,No,0.893736
2,EUDESMANE,208.389,1,180.165,11,208.219,2,-6.093,0,5.564,...,0.033,0.019,0.029,0.483,0.039,0.948,0.293,0.006,Yes,0.834177
3,orientin,448.38,3,428.22,24,448.101,4,-3.741,2,-0.067,...,0.024,0.822,0.099,0.342,0.035,0.061,0.036,0.943,Yes,0.879454
4,oxalic acid,90.034,0,88.018,2,89.9953,0,0.24,0,-0.735,...,0.024,0.026,0.038,0.504,0.014,0.994,0.068,0.005,No,0.933


In [150]:
df.to_csv("OSADHI_Chemo_all.csv", index='False')